# Portfolio Optimization

## Introduction

Consider the following scenario: Xiaoming, an astute individual, possesses a sum of money represented by $B$ and is contemplating investing it in the stock market. The market comprises of $n$ shares, each having an identical price. Xiaoming's objective is to maximize returns while minimizing risk, taking into account the varying levels of risk tolerance among individuals. Xiaoming's personal risk tolerance is represented by $p$. In light of these considerations, the question arises: which shares should Xiaoming choose to construct an optimal portfolio?

Xiaoming's predicament falls under the purview of portfolio optimization problems. These problems are classified as Quadratic Unconstrained Binary Optimization (QUBO) problems, wherein binary numbers are utilized to represent decisions. In this case, "1" signifies selection, while "0" denotes the opposite. To address the challenge of portfolio optimization, the Quantum Approximate Optimization Algorithm (QAOA) is employed.

## Solving portfolio optimization problems with QAOA

In a simple boolean Markowitz portfolio optimization problem, we wish to solve 

$$\min_{x\in\{0,1\}^n}\quad q x^T \Sigma x - \mu^T x$$

subject to a constraint

$$ 1^T x = B$$

where 
* $n$: number of assets under consideration
* $q > 0 $: risk-appetite
* $\Sigma \in \mathbb{R}^{n\times n}$: covariance matrix of the assets
* $\mu\in\mathbb{R}^n$: mean return of the assets
* $B$: budget (i.e., total number of assets out of $n$ that can be selected)

Our first step is to convert this constrained quadratic programming problem into a QUBO.  We do this by adding a penalty factor $t$ and consider the alternative problem:

$$ \min_{x\in\{0,1\}^n}\quad q x^T \Sigma x - \mu^Tx  + t(1^Tx-B)^2$$

The variables in the linear terms $\mu^Tx = \mu_1 x_1 + \mu_2 x_2+\ldots$ can all be written in a squared form, since all boolean variables $0^2=0,\ 1^2=1$. The same trick is applied on the middle term of $t(1^Tx-B)^2$. Then the function is written as

$$\min_{x\in\{0,1\}^n}\quad q x^T \Sigma x - \sum_{i=1}^n\mu_i x_i^2  + t(1^Tx-B)^2$$

which is a QUBO problem

$$\min_{x\in\{0,1\}^n}\quad x^T Q X + tB^2$$

where matrix $Q$ is

$$ Q = q\Sigma -\mu\begin{pmatrix}1 & \\ & 1\\ & & \ddots\end{pmatrix} + t\begin{pmatrix}1 -2B & 1 & \ldots & 1 \\
1 & 1-2B & 1 & \ldots \\1 & 1 & 1-2B \\
\vdots\end{pmatrix}$$

and we ignore the constant term $t B^2$. We can now solve this by QAOA as above.

Define the python class `StockData` to convert real-world stocks data into the annualized return vector and annualized covariance matrix.

In [1]:
## Ready to delete
from typing import List
from QAOA_funcs import QUBO_to_Ising, print_result_cost, print_result_prob
from QAOA_funcs import print_Q_cost

In [2]:
import tensorcircuit as tc
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from IPython.display import clear_output
from functools import partial
import time
import scipy.optimize as optimize

K = tc.set_backend("tensorflow")
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

In [3]:
class StockData:
    '''convert real-world stock data to annualized covariance matrix and annualized return
    input: a list of continuous stock data in the same time span
    output: annualized convariance matrix and return
    '''

    def __init__(self, data):
        self.data = data  # add data
        self.n_stocks = len(data)  # num of stocks

        # check the number of days
        n_days = [len(i) for i in data]
        if max(n_days) != (sum(n_days) / len(n_days)):
            raise Exception("timespan of stocks should be the same")
        self.n_days = len(data[1])

        # calculate the daily percentage price change
        self.daily_change = []  # daily percentage price change
        for i in range(self.n_stocks):
            each_stcok = []
            for j in range(self.n_days - 1):
                each_stcok.append((data[i][j + 1] - data[i][j]) / data[i][j])
            self.daily_change.append(each_stcok)

    # calculate annualized return (mu)
    def get_return(self):
        ret = np.mean(self.daily_change, axis=1)
        return ret

    # calculate annualized covariance matrix (sigma)
    def get_covariance(self):
        return np.cov(self.daily_change)

    def get_pentalty(self, cov, ret, risk_pre, budget):
        # get all fesible and unfeasible states
        self.f_state = []  # feasible states (num of '1's equal to budge)
        self.uf_state = []  # unfeasible states
        self.all_state = []
        for i in range(2**self.n_stocks):
            state = f"{bin(i)[2:]:0>{self.n_stocks}}"
            n_ones = 0
            for j in state:
                if j == "1":
                    n_ones += 1
            self.all_state.append(state)
            if n_ones == budget:
                self.f_state.append(state)
            else:
                self.uf_state.append(state)

        # determine the penalty factor
        mark = False
        penalty = 0  # initial value
        while mark == False:
            R = np.diag(ret)
            S = np.ones((self.n_stocks, self.n_stocks)) - 2 * budget * np.diag(
                np.ones(self.n_stocks)
            )
            Q = risk_pre * cov - R + penalty * S
            F = []
            for state in self.f_state:
                x = np.array([int(bit) for bit in state])
                F.append(np.dot(x, np.dot(Q, x)) + penalty * budget**2)
            Fmin = np.amin(F)
            Fbar = np.mean(F)
            F = []
            for state in self.uf_state:
                x = np.array([int(bit) for bit in state])
                F.append(np.dot(x, np.dot(Q, x)) + penalty * budget**2)
            Fmin_uf = np.amin(F)
            location = np.where(F == Fmin_uf)[0][0]
            if Fmin_uf < 0.5 * (Fmin + Fbar):
                n_ones = 0
                for j in self.uf_state[location]:
                    if j == "1":
                        n_ones += 1
                penalty += (0.5 * (Fmin + Fbar) - Fmin_uf) / (n_ones - budget) ** 2
                # mark = True
            else:
                mark = True  # ready to return the penalty
        return penalty

In [4]:
# real-world stock data, calculated using the functions above
# stock name: aapl, amzn, meta, msft, qcom, sbux
# from 10/06/2022 to 09/06/2023
mu = [-1.1734e-03, 1.2696e00, 4.3733e-01, 7.6221e-02, 2.3281e-02, 8.1355e-01]
sigma = np.array(
    [
        [3.8891e-03, -3.0224e-02, 2.0261e-03, 5.6387e-04, 8.3185e-03, 1.9666e-02],
        [-3.0224e-02, 1.0839e02, 1.3541e01, -2.3283e-01, -2.0257e-01, 1.4126e01],
        [2.0261e-03, 1.3541e01, 1.2007e01, -9.7126e-02, -4.4600e-02, 4.3367e00],
        [5.6387e-04, -2.3283e-01, -9.7126e-02, 1.4255e00, 1.6567e-02, -5.6537e-02],
        [8.3185e-03, -2.0257e-01, -4.4600e-02, 1.6567e-02, 5.3393e-02, -1.8909e-02],
        [1.9666e-02, 1.4126e01, 4.3367e00, -5.6537e-02, -1.8909e-02, 3.0492e01],
    ]
)

In [5]:
def QUBO_from_portfolio(cov, mean, q, B, t):
    """convert portfolio parameters to a Q-matrix
    cov: n-by-n covariance numpy array
    mean: numpy array of means
    q: the risk preference of investor
    B: budget
    t: penalty factor
    """
    n = cov.shape[0]
    R = np.diag(mean)
    S = np.ones((n, n)) - 2 * B * np.diag(np.ones(n))

    Q = q * cov - R + t * S
    return Q

We can test this using the qiskit_finance package to generate some stock covariance and mean data:

*Note that this was tested with qiskit version 0.39.3 and qiskit-finance version 0.3.4.*

Using this mean and covariance data, we can now define our portfolio optimization problem, convert it to a QUBO matrix, and then extract the pauli terms and weights

In [6]:
q = 0.5 # the risk preference of investor
budget = 3  # Note that in this example, there are 6 assets, but a budget of only 3
penalty = 21.848849746478102#80299.22578764349 # calculated using the function above
penalty1 = 2

#Q = QUBO_from_portfolio(sigma, mu, q, budget, penalty)
Q = QUBO_from_portfolio(sigma, mu, q, budget, penalty)
portfolio_pauli_terms, portfolio_weights, portfolio_offset = QUBO_to_Ising(Q)

In [7]:
print_Q_cost(Q, wrap=True)


-------------------------------------
    selection	  |	  cost
-------------------------------------
    100110	  |	-195.9711
    101010	  |	-191.1012
    001110	  |	-190.5587
    101100	  |	-190.5284
    100011	  |	-182.1916
    000111	  |	-181.6261
    100101	  |	-181.6039
    100010	  |	-174.7759
-------------------------------------


In [ ]:
states = []
for i in range(2**n_stocks):
    a = f"{bin(i)[2:]:0>{n_stocks}}"
    n_ones = 0
    for j in a:
        if j == '1':
            n_ones += 1
    if True:#n_ones != budget:
        states.append(a)
print(states)

In [ ]:
# Brutely search over classical results for comparison before we run QAOA
# the results are sorted with cost
cost_dict = {}
for selection in states:
    x = np.array([int(bit) for bit in selection])
    cost_dict[selection] = np.dot(x, np.dot(Q, x))
cost_sorted = dict(sorted(cost_dict.items(), key=lambda item: item[1]))
print("\n-------------------------------------")
print("    selection\t  |\t  cost")
print("-------------------------------------")
for k, v in cost_sorted.items():
    print("%10s\t  |\t%.4f" % (k, v))
print("-------------------------------------")

### use vvage

In [ ]:
iterations = 2000
nlayers = 7
time_start = time.time()
final_params = QUBO_QAOA(Q, QAOA_loss, nlayers, iterations, vvag=True, ncircuits=1000)
clear_output(wait=True)
time_end = time.time()
print("time consumed:", round(time_end - time_start, 4), "s")

In [ ]:
rank = []
for single_params in final_params:
    c_final = QAOA_from_Ising(
        single_params, nlayers, portfolio_pauli_terms, portfolio_weights
    )
    probs = K.numpy(c_final.probability()).round(decimals=6)
    sorted_indices = np.argsort(probs)[::-1]
    rank.append(np.where(sorted_indices==38)[0][0])
print(rank)

In [ ]:
good_results = [i for i in rank if i < 8]
print('the probability that the best combination shows up in top 8 is:', len(good_results)/len(rank)*100, '%')

In [ ]:
import pandas as pd
df = pd.DataFrame(rank)
df.to_csv('vvag.csv')

In [ ]:
plt.plot(rank)

We see that, due to the penalty, the lowest energy solutions correspond to 0111, 1011, 1101, 1110, i.e. the portfolios with only 3 assets.

In [ ]:
iterations = 1000
nlayers = 10
time_start = time.time()
final_params = QUBO_QAOA(Q, QAOA_loss, nlayers, iterations)
clear_output(wait=True)
time_end = time.time()
print("time consumed:", round(time_end - time_start, 4), "s")

In [ ]:
c_final = QAOA_from_Ising(
    final_params, nlayers, portfolio_pauli_terms, portfolio_weights
)
states = []
for i in range(2**n_stocks):
    a = f"{bin(i)[2:]:0>{n_stocks}}"
    states.append(a)
probs = K.numpy(c_final.probability()).round(decimals=6)
sorted_indices = np.argsort(probs)[::-1]
state_sorted = np.array(states)[sorted_indices]
prob_sorted = np.array(probs)[sorted_indices]

print("\n-------------------------------------")
print("    selection\t  |\tprobability")
print("-------------------------------------")
for i in range(len(states)):
    print("%10s\t  |\t  %.4f" % (state_sorted[i], prob_sorted[i]))
print("-------------------------------------")


In [ ]:
print_output(c_final)


### Influence of different mixers

In [ ]:
# difine an universal QAOA ansatz
def QAOA_ansatz_mixer(params, nlayers, pauli_terms, weights, mixer="standard", gap=5):
    nqubits = len(pauli_terms[0])
    c = tc.Circuit(nqubits)
    for i in range(nqubits):
        c.h(i)
    for j in range(nlayers):
        # cost term
        for k in range(len(pauli_terms)):
            term = pauli_terms[k]
            index_of_ones = []
            for l in range(len(term)):
                if term[l] == 1:
                    index_of_ones.append(l)
            if len(index_of_ones) == 1:
                c.rz(index_of_ones[0], theta=2 * weights[k] * params[2 * j])
            elif len(index_of_ones) == 2:
                c.exp1(
                    index_of_ones[0],
                    index_of_ones[1],
                    unitary=tc.gates._zz_matrix,
                    theta=weights[k] * params[2 * j],
                )
            else:
                raise ValueError("Invalid number of Z terms")

        # standard mixer term
        if mixer == "standard":
            for i in range(nqubits):
                c.rx(i, theta=params[2 * j + 1])

        # XY mixer
        elif mixer == "ring":
            for i in range(nqubits - 1):
                c.exp1(i, i + 1, unitary=tc.gates._xx_matrix, theta=params[2 * j + 1])
                c.exp1(i, i + 1, unitary=tc.gates._yy_matrix, theta=params[2 * j + 1])
            c.exp1(nqubits - 1, 0, unitary=tc.gates._xx_matrix, theta=params[2 * j + 1])
            c.exp1(nqubits - 1, 0, unitary=tc.gates._yy_matrix, theta=params[2 * j + 1])

        # parity ring mixer
        elif mixer == "par_ring":
            half = int(np.floor(nqubits / 2))
            if 2 * half < nqubits:
                end = half
            else:
                end = half - 1
            for i in range(half):  # even
                c.exp1(2 * i, 2 * i + 1, unitary=tc.gates._xx_matrix, theta=params[2 * j + 1])
                c.exp1(2 * i, 2 * i + 1, unitary=tc.gates._yy_matrix, theta=params[2 * j + 1])
            for i in range(end):  # odd
                c.exp1(2 * i + 1, 2 * i + 2, unitary=tc.gates._xx_matrix, theta=params[2 * j + 1])
                c.exp1(2 * i + 1, 2 * i + 2, unitary=tc.gates._yy_matrix, theta=params[2 * j + 1])

        # full mixer
        elif mixer == "full":
            for i in range(nqubits - gap):
                c.exp1(i, i + gap, unitary=tc.gates._xx_matrix, theta=params[2 * j + 1])
                c.exp1(i, i + gap, unitary=tc.gates._yy_matrix, theta=params[2 * j + 1])
            for i in range(gap):
                c.exp1(nqubits - gap - 1 + i, 1 + gap, unitary=tc.gates._xx_matrix, theta=params[2 * j + 1])
                c.exp1(nqubits - gap - 1 + i, 1 + gap, unitary=tc.gates._yy_matrix, theta=params[2 * j + 1])
        #
        elif mixer == "QAMPA":
            pass

        else:
            raise RuntimeError("Invalid mixer name")
    return c

In [ ]:
def QAOA_loss_mixer(nlayers, pauli_terms, weights, params):
    c = QAOA_ansatz_mixer(params, nlayers, pauli_terms, weights, 'par_ring')
    return Ising_loss(c, pauli_terms, weights)

In [ ]:
iterations = 500
time_start = time.time()
final_params = QUBO_QAOA(Q, QAOA_loss_mixer, nlayers, iterations)
clear_output(wait=True)
time_end = time.time()
print("time consumed:", round(time_end - time_start, 4), "s")

### Use CVaR

In [ ]:
def cvar_value(r, p, percent):
    rs = sorted(
        [(i, j) for i, j in enumerate(r)], key=lambda s: -s[1]
    )  # larger to smaller
    sump = 0.0  # the sum of probability
    count = 0
    cvar_result = 0.0
    while sump < percent:
        if round(sump + p[rs[count][0]], 7) >= percent:
            cvar_result += rs[count][1] * (percent - sump)
            count += 1
            break
        else:
            sump += p[rs[count][0]]
            cvar_result += rs[count][1] * p[rs[count][0]]
            count += 1

    cvar_result /= percent
    return K.real(cvar_result)

In [ ]:
def cvar_from_circuit(circuit, num_samples, Q, alpha):
    s = circuit.state()
    results = tc.quantum.measurement_results(
        s, counts=num_samples, format="count_dict_bin"
    )  # get readouts
    results = {k: v / num_samples for k, v in results.items()}
    values = []  # passed to cvar
    probability = []  # passed to cvar
    for k, v in results.items():
        x = np.array([int(bit) for bit in k])
        values.append(np.dot(x, np.dot(Q, x)))
        probability.append(v)
    cvar_result = cvar_value(values, probability, alpha)
    return cvar_result

In [ ]:
cvar_result = cvar_from_circuit(c_final, 1000, Q, 1)
print(cvar_result)


In [ ]:
def cvar_loss(nlayers, Q, pauli_terms, weights, percent, params):
    c = QAOA_from_Ising(params, nlayers, pauli_terms, weights)
    return cvar_from_circuit(c, 1000, Q, percent)


In [ ]:
def QUBO_CVaR(Q, nlayers, alpha):
    pauli_terms, weights, offset = QUBO_to_Ising(Q)
    learning_rate = 1e-2

    loss = partial(cvar_loss, nlayers, Q, pauli_terms, weights, alpha)
    # value, COBYLA
    
    f_scipy = tc.interfaces.scipy_interface(loss, shape=[2 * nlayers], jit=False, gradient=False)
    params = K.implicit_randn(shape=[2 * nlayers], stddev=0.5)
    r = optimize.minimize(f_scipy, params, method="COBYLA")

    return r

In [ ]:
def QUBO_CVaR_vvag(Q, nlayers, alpha):
    pauli_terms, weights, offset = QUBO_to_Ising(Q)
    learning_rate = 1e-2

    cvar_vvag = tc.backend.vvag(cvar_loss, argnums=0, vectorized_argnums=0)
    ncircuits = 3
    params = K.implicit_randn(
        shape=[ncircuits, 2 * nlayers], stddev=0.1
    )  # initial parameters
    loss = partial(cvar_vvag, nlayers, Q, pauli_terms, weights, alpha)
    
    f_scipy = tc.interfaces.scipy_interface(loss, shape=[ncircuits,2 * nlayers], jit=False, gradient=False)
    #params = K.implicit_randn(shape=[2 * nlayers], stddev=0.5)
    r = optimize.minimize(f_scipy, params, method="COBYLA")

    return r

In [ ]:
# iterations = 500
time_start = time.time()
nlayers = 20
final_params = QUBO_CVaR(Q, nlayers, 0.6)
clear_output(wait=True)
time_end = time.time()
print("time consumed:", round(time_end - time_start, 4), "s")

# probabilities

In [ ]:
prob = [[]]
last_prob = 0
single_prob = []
for j in range(2):
    clear_output(wait=True)
    print('times', j)
    print('last prob', last_prob)
    iterations = 1000
    nlayers = 10
    time_start = time.time()
    final_params = QUBO_QAOA(Q, QAOA_loss, nlayers, iterations)
    c_final = QAOA_from_Ising(
        final_params, nlayers, portfolio_pauli_terms, portfolio_weights
    )
    probs = K.numpy(c_final.probability()).round(decimals=6)
    single_prob.append(probs[38])
    last_prob = probs[38]
    print(single_prob)
prob.append(single_prob)
clear_output(wait=True)
print(prob)

# rankings

In [ ]:
alpha = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
total_rank = [[]]
last_rank = 0
n_layers = 15
for i in alpha:
    rank = []
    for j in range(20):
        clear_output(wait=True)
        print('alpha', i)
        print('times', j)
        print('rank', rank)
        print("total", total_rank)
        result = QUBO_CVaR(Q, nlayers, i)
        c_final = QAOA_from_Ising(
            result.x, nlayers, portfolio_pauli_terms, portfolio_weights
        )
        probs = K.numpy(c_final.probability()).round(decimals=6)
        prob35 = probs[35]
        loc = np.where(np.sort(probs)==prob35)
        rank.append(loc[0][0])
    total_rank.append(rank)
clear_output(wait=True)
print("total", total_rank)

In [ ]:
import pandas as pd
name = ['1', '2', '3', '4', '5', '6', '7', '8', '9']
df = pd.DataFrame(total_rank[1:])
df.to_csv('rank.csv')

In [ ]:
prob5 = [[round(j, 5) for j in i] for i in prob]
print(prob5)

In [ ]:
final_params.x
c_final = QAOA_from_Ising(
    final_params.x, nlayers, portfolio_pauli_terms, portfolio_weights
)
states = []
for i in range(2**n_stocks):
    a = f"{bin(i)[2:]:0>{n_stocks}}"
    states.append(a)
probs = K.numpy(c_final.probability()).round(decimals=6)
sorted_indices = np.argsort(probs)[::-1]
state_sorted = np.array(states)[sorted_indices]
prob_sorted = np.array(probs)[sorted_indices]

print("\n-------------------------------------")
print("    selection\t  |\tprobability")
print("-------------------------------------")
for i in range(len(states)):
    print("%10s\t  |\t  %.4f" % (state_sorted[i], prob_sorted[i]))
print("-------------------------------------")

# Rewrite with Qiskit

In [ ]:
from qiskit import QuantumCircuit

ansatz = 

# Benchmarking